In [ ]:
!pip install langchain
!pip install unstructured
!pip install openai
!pip install chromadb
!pip install Cython
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 88.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 62.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader, DirectoryLoader, UnstructuredPowerPointLoader, Docx2txtLoader
#from langchain.document_loaders import UnstructuredDirectoryLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#don't execute

root_dir = "/content/drive/My Drive"
pdf_folder_path = f'{root_dir}/Knowledge Base/'
#os.listdir(pdf_folder_path)
os.listdir('/content/drive/MyDrive/Knowledge Base')

['Northwind_Standard_Benefits_Details.pdf',
 'CauseAndEffectOfHomelessness.txt',
 'CauseAndEffectOfHomelessness2.txt',
 'CauseAndEffectOfHomelessness3.txt',
 'Northwind_Health_Plus_Benefits_Details (1).pdf',
 'Benefit_Options.pdf',
 'PerksPlus.pdf',
 'role_library.pdf',
 'File Viewer Migration factory.pdf']

In [ ]:
loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]

In [ ]:
text_loader = DirectoryLoader('/content/drive/MyDrive/Knowledge Base', glob='**/*.txt')
pdf_loader = DirectoryLoader('/content/drive/MyDrive/Knowledge Base', glob='**/*.pdf')
readme_loader = DirectoryLoader('/content/drive/MyDrive/Knowledge Base', glob='**/*.md')
doc_loader = DirectoryLoader('/content/drive/MyDrive/Knowledge Base', glob='**/*.docx')
ppt_loader = DirectoryLoader('/content/drive/MyDrive/Knowledge Base', glob='**/*.pptx')

In [ ]:
loaders = [pdf_loader, readme_loader, text_loader, doc_loader, ppt_loader]
documents = []
for loader in loaders:
  documents.extend(loader.load())

In [ ]:
print(f'You have {len(documents)} documents in your data')

You have 11 documents in your data


In [ ]:
#text splitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(documents)
print(len(documents))

690


Vector Store

In [ ]:

index = VectorstoreIndexCreator(vectorstore_cls=Chroma,embedding=OpenAIEmbeddings(),text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)).from_loaders(loaders)
index

VectorStoreIndexWrapper(vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x7f8ef8aff670>)

In [ ]:
#with pdf
print(index.query('What is the Northwind Standard Health Plan'))
print(index.query_with_sources('What is the Northwind Standard Health Plan')['sources'])

 The Northwind Standard Health Plan is a comprehensive health plan that provides coverage for medical, vision, and dental services, as well as preventive care services and prescription drug coverage. It offers a variety of in-network providers, including primary care physicians, specialists, hospitals, and pharmacies. It does not offer coverage for emergency services, mental health and substance abuse coverage, or out-of-network services.
/content/drive/MyDrive/Knowledge Base/Northwind_Standard_Benefits_Details.pdf, /content/drive/MyDrive/Knowledge Base/Benefit_Options.pdf, /content/drive/MyDrive/Knowledge Base/Northwind_Standard_Benefits_Details.pdf


In [ ]:
#with md
print(index.query('How to setup CMAV tool'))
print(index.query_with_sources('How to setup CMAV tool')['sources'])

 Follow the steps outlined in the context: 1. Download and unzip the file. 2. Open the “CloudMigrationAssessmentAndValidation.sln” file in Visual Studio or any other IDE. 3. Now open the “Configuration.json” file and set the values for different keys, according to the below:  - SourcePaths: This should contain a list of path(s) for the folder(s) of the source code of your project. - IgnoreCase: This is a Boolean value, if set to “true” will ignore the match case while parsing the keywords, else will not ignore the case when set to “false”. - SearchForHardCodedPath: This is a Boolean value, if set to “true” will log all the hard-coded paths/network paths, else will not in case of “false”. - FileExtensions: This contains a list of allowed file extensions, through which only the keywords will be parsed. - Required: This contains the set of keywords which must be present in a project. You can make changes to this list according to your requirements. - Validate: This contains the set of key

In [ ]:
#with txt
print(index.query('What are the effects of homelessness'))
print(index.query_with_sources('What are the effects of homelessness')['sources'])

 Poor health, personal and psychological decline/effects, decreased access to opportunity.
/content/drive/MyDrive/Knowledge Base/CauseAndEffectOfHomelessness2.txt, /content/drive/MyDrive/Knowledge Base/CauseAndEffectOfHomelessness3.txt


In [ ]:
#with doc
print(index.query('Tell me about criminal violence against Black Americans'))
print(index.query_with_sources('Tell me about criminal violence against Black Americans')['sources'])

 Criminal violence against Black Americans has been a major issue in the United States in the last two decades. The rate of violence in the Black American communities was exceedingly high, with the number of violence cases in the United States among the Black Americans exceeding the number of those in both Germany and France by twenty-eight times. In 1990, most of homicidal cases were discovered to target Black American males by the Centers for Disease Control (CDC). Additionally, the highest number of these cases involved men between the ages of fifteen and twenty-four years. The number of crime victims within this age was around one hundred and thirty two out of one hundred thousand youths. Among these crimes, most of them ended up in deaths and fatal or permanent injuries. The rate of acquisition of weapons among the black American community is the largest factor that causes the huge crime rate. In the early part of the 1980’s, most of the crimes charged for were related to the use 

In [ ]:
#with ppt
print(index.query('Problem Statement Related to Migration'))
print(index.query_with_sources('Problem Statement Related to Migration')['sources'])

 The problem statement related to migration is that over a decade there have been many artifacts built to support the growing need of standard operating procedures, tools, user guides, checklists/playbooks, automations, technical trackers, and learnings. As an initial prototype, we are planning to build a ChatGPT kind of solution to help Factory team members to identify the artifacts and information that would enable them to consume the info more efficiently and will optimize their delivery.
/content/drive/MyDrive/Knowledge Base/OPEN AI-BASED FILE RETRIEVAL SYSTEM (1).pptx
